# Titanic – Machine Learning from the disaster

First, we import these libraries below to initialize our project.

In [4]:
# Data manipulation
import numpy as np
import pandas as pd

# Data visualizzation
import matplotlib.pyplot as plt
import missingno
import seaborn as sns

ModuleNotFoundError: No module named 'scipy._lib'

## Explore the data

Load the data from the CSV file and understand the structure

In [ ]:
train = pd.read_csv("data/train.csv")
print(f"Training set: {train.shape}")
train.head()

In [ ]:
test = pd.read_csv("data/test.csv")
print(f"Test set: {test.shape}")
test.head()

### Data description

- **survival**: 0 = No, 1 = Yes
- **pclass**: Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd
- **sex**: male or female
- **age**: age in years, is fractional if less than 1
- **sibsp**: the number of siblings or spouses onboard
- **parch**: the number of parents or children onboard
- **ticket**: ticket numbers
- **fare**: passenger fare
- **cabin**: cabin number
- **embarked**: port of embarkation, where C = Cherbourg, Q = Queenstown, S = Southampton

### Missing data

Which columns in the dataset have missing data?

In [ ]:
# Columns with missing data in training set
missingno.matrix(train, figsize=(15,10))


In [ ]:
# Columns with missing data in test set
missingno.matrix(test, figsize=(15,10))

## Data analysis

We will epxlore each variable in the data set. This may help us to choose the suitable machine learning models, or whether we need to determine the dataset.

### Categorical variables

- Nominal variables: **Sex**, **Emarked**
- Oridinal variables: **Pclass**

In [ ]:
# Get the frequeny of each nominal variables
cat_features = ["Sex", "Embarked", "Pclass"]
f, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, feature in enumerate(cat_features):
    sns.countplot(x=feature, data=train, ax=axes[i])

#### Insights from counting plots

- There are more male passengers than female passengers
- Most passengers were departed from Southampton
- Most passengers were holding third-class tickets

In [ ]:
# What is the survival rate based on each categorical variables
f, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, feature in enumerate(cat_features):
    sns.barplot(x=feature, y="Survived", data=train, ax=axes[i])
    axes[i].set_ylabel("Survival rate")


#### Insights about survival plots

- Female passengers were more likely to survive than male passengers. This may suggests that female passengers were prioritised to evacuate.
- Passengers from Cherbougs were the most likely to survive, while those from Southampton were the least.
- The higher the ticket classes were, the more likely the passengers to survived. This may suggest evacuation priority were based on the ticket class (first-class passengers first, thrid-class passengers last).

### Numerical variables

#### Outliers

Outliers are extreme values which do not conform the majority, and contribute the the skewness of the data set. Having outlier in the dataset may results in learning models with low accuracy. Therefore, we need to filter out these outliers for better models.

For this dataset, the filter is based on interquatile range ($IQR$), which means we will keep the numerical values when they are in range of

$$[Q_1 - 1.5 \cdot (Q_3 - Q_1), Q_3 + 1.5 \cdot (Q_3 - Q_1)]$$

where $Q_1$ is 25th percentile, $Q_3$ is 75th percentile

In [ ]:
print(f"Before: {len(train)} rows")

def outliersIndices():
  numerical = train[["SibSp", "Parch", "Age", "Fare"]]
  q1 = numerical.quantile(.25)
  q3 = numerical.quantile(.75)
  iqr = q3 - q1
  low = q1 - 1.5 * iqr
  high = q3 + 1.5 * iqr
  # Only filter out the data if it has more than 2 outlier fields
  outlierCount = (~numerical.isnull() & ((numerical < low) | (numerical > high))).sum(axis=1)
  return outlierCount[outlierCount > 2].index
  
indices = outliersIndices()
print(f"Remove rows: {list(indices)}")

# Filter out
train = train.drop(indices, axis=0).reset_index(drop=True)
print(f"After: {len(train)} rows")


#### Correlation

Some numerical variables may correlate with the survival state

In [ ]:
train[["SibSp", "Parch", "Age", "Fare"]].corrwith(train["Survived"])

# Comment: There is some positive correlation between the fare and survival.

#### Discrete variables

`SibSp` and `Parch` are discrete variables

In [ ]:
# Counting plots for discrete variables
f, axes = plt.subplots(1, 2, figsize=(15, 5))
discrete = ["SibSp", "Parch"]
for i, feature in enumerate(discrete):
    sns.countplot(x=feature, data=train, ax=axes[i])

In [ ]:
# What are the survival rates for pasenger based on their companions
f, axes = plt.subplots(1, 2, figsize=(15, 5))
for i, feature in enumerate(discrete):
    sns.barplot(x=feature, y="Survived", data=train, ax=axes[i])
    axes[i].set_ylabel("Survival rate")

#### Continuous variables

`Age` and `Fare` are continuous variables in the dataset.

In [ ]:
# What is the distribution for the countinuous variables?
f, axes = plt.subplots(1, 2, figsize=(15, 5))
continuous = ["Age", "Fare"]
for i, feature in enumerate(continuous):
    sns.histplot(train[feature], ax=axes[i], kde=True)
    axes[i].annotate("Skewness {:.2f}".format(train[feature].skew()), (0.5,0.5), xycoords="axes fraction")
    axes[i].set_title(f"{feature} distribution")

##### Insights from age and fare distribution

- Age distribution is slightly left-skewed, where younger passengers are conrtbituted significantly to the dataset
- Fare distribution is heavily skewed to the left, which conforms the distribution of ticket classes mentioned in the [catergorical variables analysis](#categorical-variables). Data transformation such as logarithmic transformation is needed in order to reblance the datadistribution

In [ ]:
# What are the age and fare distributions of survivors and non-sourvivors?
f, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(train["Age"][train["Survived"] == 1], kde=True, stat="density", ax=axes[0])
axes[0].set_title("Age distribution of survivors")
sns.histplot(train["Age"][train["Survived"] == 0], kde=True, stat="density", ax=axes[1])
axes[1].set_title("Age distribution of non-survivors")

##### Insights from survival age distribution

- Children or passengers of young age are more likely to survive. This may suggest that chidlren were more priorised for evacuation, along with women as mentioned before.

## Data proprocessing

### Feature engineering: `Cabin`, `Ticket`, and `Name`

Since `Cabin`, `Ticket`, and `Name` have too many unqiue values, we have to apply data transformation to extract some useful patterns to cut down the number of categorical values.

#### New feature from `Cabin`: `Deck`

The `Cabin` value is consisted of a letter follow by a number. We'll only extract the letter since it indicates the deck of the Titanic ships, which may have affect on the survival rate of the passengers

In [ ]:
# Extract a letter from the cabin data
# n = null, passenger without a cabin
def extractDeck(data):
  data["Deck"] = data["Cabin"].apply(lambda x: 'n' if pd.isnull(x) else str(x)[0])
  data.drop("Cabin", axis=1, inplace=True)

extractDeck(train)
extractDeck(test)

# Count the frequency
f, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.countplot(x="Deck", data=train, ax=axes[0])

# What is the survival rate based on section?
sns.barplot(x="Deck", y="Survived", data=train, ax=axes[1])
axes[1].set_ylabel("Survival rate")

#### `Ticket`

Since the ticket values generally have letters and numbers. However, since they have many different formats, we cannot find useful pattern in the dataset. Therfore, `Ticket` was dropped from the data set

In [ ]:
train.drop("Ticket", axis=1, inplace=True)
test.drop("Ticket", axis=1, inplace=True)

#### New feature from `Name`: `NameTitle`

By exploring the names of the passenger, we found that most of them contains a title (such as Mr. or Mrs.)

In [ ]:
# Extract the name title
def extractTitle(data):
  data["NameTitle"] = train["Name"].apply(lambda name: name.split(',')[1].split('.')[0].strip())
  data.drop("Name", axis=1, inplace=True)

extractTitle(test)
extractTitle(train)


# What is the title distribution of name title in training set?
pd.concat([train["NameTitle"], test["NameTitle"]]).value_counts().sort_values(ascending=False)

In [ ]:
# Clean and group titles together
# Mme -> Mrs
# Ms, Mlle -> Miss
titleMap = {
  "Mme": "Mrs", "Mrs": "Mrs", "Mr": "Mr", "Master": "Master",
  "Miss": "Miss", "Ms": "Miss", "Mlle": "Miss",
}
def cleanTitle(title: str):
  if title in titleMap:
    return titleMap[title]
  return "Other"

# Now, we only have 5 cateogries for name title: Mr, Mrs, Miss, Master, Other
train["NameTitle"] = train["NameTitle"].apply(cleanTitle)
test["NameTitle"] = test["NameTitle"].apply(cleanTitle)

# What is the distrubution and the survival rate?
f, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.countplot(x="NameTitle", data=train, ax=axes[0])
axes[0].set_xlabel("Name Title")

sns.barplot(x="NameTitle", y="Survived", data=train, ax=axes[1])
axes[1].set_xlabel("Name Title")
axes[1].set_ylabel("Survival rate")

### Filling missing data

In [ ]:
# Missing data from training set
missing = train.isnull().sum()
missing[missing > 0].sort_values(ascending = False)

In [ ]:
# Missing data from test set
missing = test.isnull().sum()
missing[missing > 0].sort_values(ascending = False)

For the `Embarked` in the training set, the missing data is filled with the most common one

In [ ]:
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

For the `Fare` in the test set, the missing data is filled with the median

In [ ]:
test["Fare"].fillna(test["Fare"].dropna().median(), inplace=True)

#### Filling `Age` info

As mentioned before, there are signifincant portion of  passengers without age info.

From the result below, `Pclass`, `Parch`, and `SibSp` are somewhat correlation with `Age`. We can use this information to fill the missing `Age` data

In [ ]:
# Binary encoding for sex
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# What is the correlation for between categorical and discrete variable and the age?
train[["Sex", "Pclass", "SibSp", "Parch"]].corrwith(train["Age"])

In [ ]:
# How about in the test set?
test[["Sex", "Pclass", "SibSp", "Parch"]].corrwith(test["Age"])

From both correlation tables above, we found that the `Age` is most correlated to the `Pclass`. We'll that information to fill the missing `Age` data

In [ ]:
# Find the rows with with missing Age
def fillAge(data):
  missingAgeRows = data[data["Age"].isnull()].index.tolist()
  meadianAge = data["Age"].dropna().median()

  for i in missingAgeRows:
    # Find the rows with the same Pclass
    predictedAge = data[data["Pclass"] == data.loc[i ,"Pclass"]]["Age"].median()
    if pd.isnull(predictedAge):
      data.loc[i, "Age"] = meadianAge
    else:
      data.loc[i, "Age"] = predictedAge

fillAge(train)
fillAge(test)

### Categorical data encoding

From the current state of the dataset, `Sex` are `Pclass` are previously encoded with numerical values.

Since `NameTitle`, `Deck`, and `Embarked` are nomimal values where each of them has a small number categories, we'll use **Dummy Encoding** scheme for these variables.


In [ ]:
# Combine the remaining categorical data
categorical = ["NameTitle", "Deck", "Embarked"]
categoricalWithId = ["PassengerId"] + categorical
combinedCat = pd.concat([train[categoricalWithId], test[categoricalWithId]], ignore_index=True)
combinedCat.head()

In [ ]:
# Dummy encoding
dummies = pd.get_dummies(combinedCat[categorical])
dummies = pd.concat([combinedCat["PassengerId"], dummies], axis=1)
dummies.head()

In [ ]:
# Merge dummy table to the data set
train = train.drop(categorical, axis=1).merge(dummies, "left", "PassengerId")
test = test.drop(categorical, axis=1).merge(dummies, "left", "PassengerId")
train.head()

### Numerical data encoding

In [ ]:
# Combine age and fare data from the training and test set
numerical = ["Age", "Fare"]
numericalWithId = ["PassengerId"] + numerical
combinedNum = pd.concat([train[numericalWithId], test[numericalWithId]], ignore_index=True)
combinedNum.head()

#### `Age` encoding

As mentioned before, the age distribution is quite normal. Thus, we can directly encode it by transform it into ordinal values

In [ ]:
combinedNum["AgeClass"] = pd.cut(combinedNum["Age"], 5, labels=range(5))
combinedNum.drop(["Age"], axis=1, inplace=True)

#### `Fare` encoding

As mentioned before, the distribution of `Fare` is heavily skewed to the left. This suggests we should perform data transformation on this data before encoding it. In this case, we choose logarithmic transformation

In [ ]:
# Logarithmic transformation
combinedNum["LogFare"] = np.log(combinedNum["Fare"] + 1)

# What is the distribution?
print(f"Skewness: {combinedNum['LogFare'].skew():.2f}")
sns.histplot(combinedNum["LogFare"], kde=True)

In [ ]:
# Encode to ordinal values
combinedNum["FareClass"] = pd.cut(combinedNum["LogFare"], 6, labels=range(6))

# Drop the temporary columns
combinedNum.drop(["Fare", "LogFare"], axis=1, inplace=True)

# What is the result look like
combinedNum.head()

In [ ]:
# Merge the new encoded data back to the original set
train = train.drop(numerical, axis=1).merge(combinedNum, "left", "PassengerId")
test = test.drop(numerical, axis=1).merge(combinedNum, "left", "PassengerId")

## Final data before modelling

Let's see the structure of two data sets

In [ ]:
# What does training set look like?
print(f"Training set: {train.shape}")
train.head()

In [ ]:
# What does test set look like?
print(f"Test set: {test.shape}")
test.head()

## Machine learning

First, import libraries for machine learning models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import KFold

### Split the data

First, we eill build the data set to input the learning models

In [ ]:
X = train.drop(["PassengerId", "Survived"], axis=1)
y = train["Survived"]
X_test = train.drop("PassengerId", axis=1)

### Learning models

- Random Forest

In [ ]:
# Random Forest
randomForest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
randomForest.fit(X, y)
score = randomForest.score(X, y)
print(f"Accuracy {score:.4f}")

- K-NN

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
score = neigh.score(X, y)
print(f"Accuracy {score:.4f}")

- Logistic Regression

In [ ]:
reg = LogisticRegression()
reg.fit(X, y)
score = reg.score(X, y)
print(f"Accuracy {score:.4f}")

- Support Vector Machine

In [ ]:
clf = svm.SVC()
clf.fit(X, y)
score = clf.score(X, y)
print(f"Accuracy {score:.4f}")

K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
splitIndices = kf.split(train)

def kFoldLearning(model):
    scores = []
    for train_index, test_index in splitIndices:
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
    print(f"Average accuracy: {sum(scores) / len(scores):.4f}")